In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, percentileofscore
import math
import os
pd.set_option('display.max_columns', None)

In [9]:
BASE_DIR = os.path.join('..', 'data')
RAW_DIR = os.path.join(BASE_DIR, 'raw')
ENGINEERED_DIR = os.path.join(BASE_DIR, 'engineered')
SEASON_DIR = os.path.join(ENGINEERED_DIR, '2023')
PLAYERS_DIR = os.path.join(SEASON_DIR, 'players_stats')

In [27]:
gk = pd.read_csv(os.path.join(PLAYERS_DIR, 'goalkeeping.csv'))
gk_adv = pd.read_csv(os.path.join(PLAYERS_DIR, 'advanced goalkeeping.csv'))
columns_to_keep_df = ['Rk',
 'Player',
 'Nation',
 'Pos',
 'Squad',
 'Comp',
 'Age',
 'Born',
 'Player Link_',
 'Player ID_',
 'Competition',
 '90s'
]
gk.drop(columns_to_keep_df + ['Playing Time_MP',
 'Playing Time_Starts',
 'Playing Time_Min',
 ], axis=1, inplace=True)
gk_adv.drop(columns_to_keep_df, axis=1, inplace=True)

In [28]:
gk_adv.columns

Index(['Goals_GA', 'Goals_PKA', 'Goals_FK', 'Goals_CK', 'Goals_OG',
       'Expected_PSxG', 'Expected_PSxG/SoT', 'Expected_PSxG+/-',
       'Expected_/90', 'Launched_Cmp', 'Launched_Att', 'Launched_Cmp%',
       'Passes_Att (GK)', 'Passes_Thr', 'Passes_Launch%', 'Passes_AvgLen',
       'Goal Kicks_Att', 'Goal Kicks_Launch%', 'Goal Kicks_AvgLen',
       'Crosses_Opp', 'Crosses_Stp', 'Crosses_Stp%', 'Sweeper_#OPA',
       'Sweeper_#OPA/90', 'Sweeper_AvgDist'],
      dtype='object')

In [10]:
df = pd.read_parquet(os.path.join(PLAYERS_DIR, 'merged_df.parquet'))
df.head(5)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,Performance_G+A,Performance_G-PK,Performance_PK,Performance_PKatt,Performance_CrdY,Performance_CrdR,Expected_xG,Expected_npxG,Expected_xAG,Expected_npxG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Player Link_,Player ID_,Competition,90s,Blocks_Blocks,Blocks_Pass,Blocks_Sh,Challenges_Att,Challenges_Lost,Challenges_Tkl,Challenges_Tkl%,Clr,Err,Int,Tackles_Att 3rd,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Tkl,Tackles_TklW,Tkl+Int,GCA Types_Def,GCA Types_Fld,GCA Types_PassDead,GCA Types_PassLive,GCA Types_Sh,GCA Types_TO,GCA_GCA,GCA_GCA90,SCA Types_Def,SCA Types_Fld,SCA Types_PassDead,SCA Types_PassLive,SCA Types_Sh,SCA Types_TO,SCA_SCA,SCA_SCA90,Aerial Duels_Lost,Aerial Duels_Won,Aerial Duels_Won%,Performance_2CrdY,Performance_Crs,Performance_Fld,Performance_Fls,Performance_Int,Performance_OG,Performance_Off,Performance_PKcon,Performance_PKwon,Performance_Recov,Performance_TklW,Att,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Outcomes_Blocks,Outcomes_Cmp,Outcomes_Off,Pass Types_CK,Pass Types_Crs,Pass Types_Dead,Pass Types_FK,Pass Types_Live,Pass Types_Sw,Pass Types_TB,Pass Types_TI,1/3,Ast,CrsPA,Expected_A-xAG,Expected_xA,KP,Long_Att,Long_Cmp,Long_Cmp%,Medium_Att,Medium_Cmp,Medium_Cmp%,PPA,PrgP,Short_Att,Short_Cmp,Short_Cmp%,Total_Att,Total_Cmp,Total_Cmp%,Total_PrgDist,Total_TotDist,xAG,Playing Time_Min%,Playing Time_Mn/MP,Starts_Compl,Starts_Mn/Start,Starts_Starts,Subs_Mn/Sub,Subs_Subs,Subs_unSub,Team Success (xG)_On-Off,Team Success (xG)_onxG,Team Success (xG)_onxGA,Team Success (xG)_xG+/-,Team Success (xG)_xG+/-90,Team Success_+/-,Team Success_+/-90,Team Success_On-Off,Team Success_PPM,Team Success_onG,Team Success_onGA,Carries_1/3,Carries_CPA,Carries_Carries,Carries_Dis,Carries_Mis,Carries_PrgC,Carries_PrgDist,Carries_TotDist,Receiving_PrgR,Receiving_Rec,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Touches_Att 3rd,Touches_Att Pen,Touches_Def 3rd,Touches_Def Pen,Touches_Live,Touches_Mid 3rd,Touches_Touches,Expected_G-xG,Expected_np:G-xG,Expected_npxG/Sh,Standard_Dist,Standard_FK,Standard_G/Sh,Standard_G/SoT,Standard_Gls,Standard_PK,Standard_PKatt,Standard_Sh,Standard_Sh/90,Standard_SoT,Standard_SoT%,Standard_SoT/90,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Expected_/90,Expected_PSxG,Expected_PSxG+/-,Expected_PSxG/SoT,Goal Kicks_Att,Goal Kicks_AvgLen,Goal Kicks_Launch%,Goals_CK,Goals_FK,Goals_GA,Goals_OG,Goals_PKA,Launched_Att,Launched_Cmp,Launched_Cmp%,Passes_Att (GK),Passes_AvgLen,Passes_Launch%,Passes_Thr,Sweeper_#OPA,Sweeper_#OPA/90,Sweeper_AvgDist,Penalty Kicks_PKA,Penalty Kicks_PKatt,Penalty Kicks_PKm,Penalty Kicks_PKsv,Penalty Kicks_Save%,Performance_CS,Performance_CS%,Performance_D,Performance_GA,Performance_GA90,Performance_L,Performance_Save%,Performance_Saves,Performance_SoTA,Performance_W
0,1,Max Aarons,ENG,DF,Bournemouth,eng Premier League,24-004,2000.0,13,11,995,11.1,0.00,0.09,0.09,0.00,0.0,0.0,0.09,0.00,0.00,0.00,0.07,0.07,1.53,3.24,1.89,0.00,0.09,0.09,0.00,0.09,0.00,0.07,0.07,0.00,0.07,https://fbref.com/en/players/774cf58b/Max-Aarons,774cf58b,Big 5 combined,11.1,0.81,0.36,0.45,2.52,0.99,1.53,60.7,1.89,0.0,0.45,0.18,1.53,0.54,2.25,1.44,2.70,0.0,0.0,0.0,0.09,0.0,0.0,0.09,0.09,0.0,0.27,0.18,1.26,0.00,0.0,1.71,1.72,0.81,0.36,30.8,0.00,0.90,1.98,0.90,0.45,0.0,0.18,0.09,0.0,5.77,1.44,42.3,0.00,0.00,0.0,1.53,32.4,0.09,0.00,0.90,9.64,0.90,32.6,0.18,0.18,8.74,1.80,0.09,0.18,0.02,0.08,0.54,4.59,2.61,56.9,17.10,13.40,78.4,0.99,3.24,18.1,15.9,87.6,42.3,32.4,76.6,204.3,536.1,0.07,56.3,77.0,0.72,86.0,0.99,25.0,0.18,0.09,-1.93,1.03,2.08,-1.06,-1.06,-1.44,-1.45,-2.50,0.69,0.90,2.34,0.99,0.54,26.6,0.54,0.81,1.53,85.4,164.1,1.89,26.0,2.43,0.90,37.0,0.81,33.

In [29]:
columns_to_drop_1 = list(gk.columns)
columns_to_drop_2 = list(gk_adv)
columns_to_drop = columns_to_drop_1 + columns_to_drop_2
columns_to_drop

['Performance_GA',
 'Performance_GA90',
 'Performance_SoTA',
 'Performance_Saves',
 'Performance_Save%',
 'Performance_W',
 'Performance_D',
 'Performance_L',
 'Performance_CS',
 'Performance_CS%',
 'Penalty Kicks_PKatt',
 'Penalty Kicks_PKA',
 'Penalty Kicks_PKsv',
 'Penalty Kicks_PKm',
 'Penalty Kicks_Save%',
 'Playing Time_90s',
 'Goals_GA',
 'Goals_PKA',
 'Goals_FK',
 'Goals_CK',
 'Goals_OG',
 'Expected_PSxG',
 'Expected_PSxG/SoT',
 'Expected_PSxG+/-',
 'Expected_/90',
 'Launched_Cmp',
 'Launched_Att',
 'Launched_Cmp%',
 'Passes_Att (GK)',
 'Passes_Thr',
 'Passes_Launch%',
 'Passes_AvgLen',
 'Goal Kicks_Att',
 'Goal Kicks_Launch%',
 'Goal Kicks_AvgLen',
 'Crosses_Opp',
 'Crosses_Stp',
 'Crosses_Stp%',
 'Sweeper_#OPA',
 'Sweeper_#OPA/90',
 'Sweeper_AvgDist']

In [30]:
df_without_gk_stats = df.drop(columns_to_drop, axis=1).copy()
df_without_gk_stats

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Performance_Gls,Performance_Ast,Performance_G+A,Performance_G-PK,Performance_PK,Performance_PKatt,Performance_CrdY,Performance_CrdR,Expected_xG,Expected_npxG,Expected_xAG,Expected_npxG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Player Link_,Player ID_,Competition,90s,Blocks_Blocks,Blocks_Pass,Blocks_Sh,Challenges_Att,Challenges_Lost,Challenges_Tkl,Challenges_Tkl%,Clr,Err,Int,Tackles_Att 3rd,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Tkl,Tackles_TklW,Tkl+Int,GCA Types_Def,GCA Types_Fld,GCA Types_PassDead,GCA Types_PassLive,GCA Types_Sh,GCA Types_TO,GCA_GCA,GCA_GCA90,SCA Types_Def,SCA Types_Fld,SCA Types_PassDead,SCA Types_PassLive,SCA Types_Sh,SCA Types_TO,SCA_SCA,SCA_SCA90,Aerial Duels_Lost,Aerial Duels_Won,Aerial Duels_Won%,Performance_2CrdY,Performance_Crs,Performance_Fld,Performance_Fls,Performance_Int,Performance_OG,Performance_Off,Performance_PKcon,Performance_PKwon,Performance_Recov,Performance_TklW,Att,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Outcomes_Blocks,Outcomes_Cmp,Outcomes_Off,Pass Types_CK,Pass Types_Crs,Pass Types_Dead,Pass Types_FK,Pass Types_Live,Pass Types_Sw,Pass Types_TB,Pass Types_TI,1/3,Ast,CrsPA,Expected_A-xAG,Expected_xA,KP,Long_Att,Long_Cmp,Long_Cmp%,Medium_Att,Medium_Cmp,Medium_Cmp%,PPA,PrgP,Short_Att,Short_Cmp,Short_Cmp%,Total_Att,Total_Cmp,Total_Cmp%,Total_PrgDist,Total_TotDist,xAG,Playing Time_Min%,Playing Time_Mn/MP,Starts_Compl,Starts_Mn/Start,Starts_Starts,Subs_Mn/Sub,Subs_Subs,Subs_unSub,Team Success (xG)_On-Off,Team Success (xG)_onxG,Team Success (xG)_onxGA,Team Success (xG)_xG+/-,Team Success (xG)_xG+/-90,Team Success_+/-,Team Success_+/-90,Team Success_On-Off,Team Success_PPM,Team Success_onG,Team Success_onGA,Carries_1/3,Carries_CPA,Carries_Carries,Carries_Dis,Carries_Mis,Carries_PrgC,Carries_PrgDist,Carries_TotDist,Receiving_PrgR,Receiving_Rec,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Touches_Att 3rd,Touches_Att Pen,Touches_Def 3rd,Touches_Def Pen,Touches_Live,Touches_Mid 3rd,Touches_Touches,Expected_G-xG,Expected_np:G-xG,Expected_npxG/Sh,Standard_Dist,Standard_FK,Standard_G/Sh,Standard_G/SoT,Standard_Gls,Standard_PK,Standard_PKatt,Standard_Sh,Standard_Sh/90,Standard_SoT,Standard_SoT%,Standard_SoT/90
0,1,Max Aarons,ENG,DF,Bournemouth,eng Premier League,24-004,2000.0,13,11,995,0.00,0.09,0.09,0.00,0.0,0.0,0.09,0.00,0.00,0.00,0.07,0.07,1.53,3.24,1.89,0.00,0.09,0.09,0.00,0.09,0.00,0.07,0.07,0.00,0.07,https://fbref.com/en/players/774cf58b/Max-Aarons,774cf58b,Big 5 combined,11.1,0.81,0.36,0.45,2.52,0.99,1.53,60.7,1.89,0.00,0.45,0.18,1.53,0.54,2.25,1.44,2.70,0.0,0.00,0.00,0.09,0.0,0.0,0.09,0.09,0.00,0.27,0.18,1.26,0.00,0.00,1.71,1.72,0.81,0.36,30.8,0.00,0.90,1.98,0.90,0.45,0.0,0.18,0.09,0.0,5.77,1.44,42.3,0.00,0.00,0.0,1.53,32.4,0.09,0.00,0.90,9.64,0.90,32.6,0.18,0.18,8.74,1.80,0.09,0.18,0.02,0.08,0.54,4.59,2.61,56.9,17.10,13.40,78.4,0.99,3.24,18.10,15.90,87.6,42.3,32.4,76.6,204.30,536.1,0.07,56.3,77.0,0.72,86.0,0.99,25.0,0.18,0.09,-1.93,1.03,2.08,-1.06,-1.06,-1.44,-1.45,-2.50,0.69,0.90,2.34,0.99,0.54,26.6,0.54,0.81,1.53,85.4,164.1,1.89,26.0,2.43,0.90,37.0,0.81,33.3,11.10,0.90,18.90,2.88,51.6,22.3,51.6,0.00,0.00,0.02,23.9,0.00,0.00,NaN,0.00,0.0,0.0,0.09,0.09,0.00,0.0,0.00
1,2,Brenden Aaronson,USA,MF,Union Berlin,de Bundesliga,23-078,2000.0,11,4,327,0.00,0.00,0.00,0.00,0.0,0.0,0.56,0.28,0.03,0.03,0.08,0.11,1.11,2.50,5.00,0.00,0.00,0.00,0.00,0.00,0.04,0.07,0.11,0.04,0.11,https://fbref.com/en/players/5bc43860/Brenden-...,5bc43860,Big 5 combined,3.6,2.50,2.50,0.00,1.94,0.56,1.39,71.4,0.28,0.00,0.00,0.28,0.83,1.11,2.22,1.67,2.22,0.0,0.00,0.00,0.28,0.0,0.0,0.28,0.28,0.00,0.00,0.28,1.94,0.00,0.00,2.22,2.20,1.11,0.28,20.0,0.28,2.22,0.83,0.83,0.00,0.0,0.00,0.00,0.0,5

In [12]:
df["Pos"].unique()

array(['DF', 'MF', 'FW', 'MF,FW', 'FW,MF', 'MF,DF', 'DF,MF', 'GK',
       'DF,FW', 'FW,DF', None], dtype=object)